In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('usdf_efd')

In [ ]:
# Now let's try to expand it to the whole run
#start = Time("2023-01-26T06:51:00", scale='utc')
#end = Time("2023-01-26T09:50:00", scale='utc')
# Second more recent run
start = Time("2023-03-23T06:50:00", scale='utc')
end = Time("2023-03-23T09:15:00", scale='utc')
# Accel correlation
#start = Time("2023-03-25T01:26:29", scale='utc')
#end = Time("2023-03-25T01:46:02", scale='utc')

az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['*'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['*'],  start, end)    

az_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['azimuth', 'taiTime'],  start, end)
el_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['elevation', 'taiTime'],  start, end)   
print(len(az_track), len(el_track))

In [ ]:
el['actualPosition'].plot()

In [ ]:
azPos = await client.select_time_series('lsst.sal.MTMount.logevent_azimuthInPosition', \
                                            ['inPosition', 'private_kafkaStamp'],  start, end)
azPos = azPos[azPos['inPosition']] # Select only the True values
elPos = await client.select_time_series('lsst.sal.MTMount.logevent_elevationInPosition', \
                                            ['inPosition', 'private_kafkaStamp'],  start, end)
elPos = elPos[elPos['inPosition']] # Select only the True values
azSlew = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['trackId', 'private_kafkaStamp'],  start, end)
print(len(azPos), len(elPos), len(azSlew))

In [ ]:
# Find all of the time stamps

# Start with start_slew times
# Better method to find the slew start

targets = azSlew.values[:,0]
times = azSlew.values[:,1]
start_slew_times_1 = []
slew_times_1 = []
for i in range(1,len(targets)):
    if targets[i] != targets[i-1]:
        start_slew_times_1.append(times[i])
print(len(start_slew_times_1))

# Now in position timestamps

inPos_1 = []
azPosValues = azPos.values[:,1]
elPosValues = elPos.values[:,1]

# Subtract 1 second from the inPos values, because the mount has stabilized well before the inPos signal
for i in range(len(azPos)):
    if azPosValues[i] > elPosValues[i]:
        inPos_1.append(azPosValues[i])
    else:
        inPos_1.append(elPosValues[i])

print(len(inPos_1))

# Now pair them up

pairMin = 1.0
pairMax = 6.0

start_slew_times = []
slew_times = []
inPos = []

for i in range(len(start_slew_times_1)):
    for j in range(len(inPos_1)):
        deltaT = inPos_1[j] - start_slew_times_1[i] 
        if deltaT > pairMin and deltaT < pairMax:
            inPos.append(inPos_1[j])
            start_slew_times.append(start_slew_times_1[i])
            slew_times.append(deltaT)
        
print(len(inPos), len(start_slew_times), len(slew_times))

In [ ]:
for index in range(len(inPos)):
    start_azs = az[(az['timestamp'] > (start_slew_times[index]-0.2)) & (az['timestamp'] < ((start_slew_times[index]-0.1)))]
    end_azs = az[(az['timestamp'] > (inPos[index]-0.2)) & (az['timestamp'] < ((inPos[index]-0.1)))]
    start_els = el[(el['timestamp'] > (start_slew_times[index]-0.2)) & (el['timestamp'] < ((start_slew_times[index]-0.1)))]
    end_els = el[(el['timestamp'] > (inPos[index]-0.2)) & (el['timestamp'] < ((inPos[index]-0.1)))]
    ip_time = Time(inPos[index], format='unix_tai', scale='utc').isot
    timestamp = ip_time.split('.')[0].replace('-','').replace(':','')
    start_az = start_azs['actualPosition'].values[0]
    end_az = end_azs['actualPosition'].values[0]
    start_el = start_els['actualPosition'].values[0]
    end_el = end_els['actualPosition'].values[0]
    delta_el = abs(end_el - start_el)
    el_value = end_els['actualPosition'].values[0]
    delta_az = abs(end_az - start_az)
    print(f"{index}, {timestamp}, Elevation={el_value:.2f}, Delta_El={delta_el:.2f}, Delta_Az={delta_az:.2f}")

In [ ]:
# Plotting the mount plots

deltaAzs = []
deltaEls = []
Els = []
InPoss = []
smoothingFactor = 0.2 # In spline creation
kernel_size = 100 # In convolution
kernel = np.ones(kernel_size) / kernel_size

fig = plt.figure(figsize = (8,8))
for index in range(len(inPos)):
    
    # Calculating the magnitude of the slew
    start_azs = az[(az['timestamp'] > (start_slew_times[index]-0.2)) & (az['timestamp'] < ((start_slew_times[index]-0.1)))]
    end_azs = az[(az['timestamp'] > (inPos[index]-0.2)) & (az['timestamp'] < ((inPos[index]-0.1)))]
    start_els = el[(el['timestamp'] > (start_slew_times[index]-0.2)) & (el['timestamp'] < ((start_slew_times[index]-0.1)))]
    end_els = el[(el['timestamp'] > (inPos[index]-0.2)) & (el['timestamp'] < ((inPos[index]-0.1)))]
    ip_time = Time(inPos[index], format='unix_tai', scale='utc').isot
    timestamp = ip_time.split('.')[0].replace('-','').replace(':','')
    start_az = start_azs['actualPosition'].values[0]
    end_az = end_azs['actualPosition'].values[0]
    start_el = start_els['actualPosition'].values[0]
    end_el = end_els['actualPosition'].values[0]
    delta_el = abs(end_el - start_el)
    el_value = end_els['actualPosition'].values[0]
    delta_az = abs(end_az - start_az)
    deltaAzs.append(delta_az)
    deltaEls.append(delta_el)
    
    plotAz = az[(az['timestamp'] > (start_slew_times[index]-2.0)) & (az['timestamp'] < (inPos[index]+2.0))]
    plotEl = el[(el['timestamp'] > (start_slew_times[index]-2.0)) & (el['timestamp'] < (inPos[index]+2.0))]
    
    ss_time = Time(start_slew_times[index], format='unix_tai', scale='utc').isot
    ip_time = Time(inPos[index], format='unix_tai', scale='utc').isot
    
    # Now calculates the spline fit and differentiate it to get the acceleration and jerk
    azPs = plotAz['actualPosition'].values
    azVs = plotAz['actualVelocity'].values
    azXs = plotAz['timestamp'].values - plotAz['timestamp'].values[0] - 2.0 
    elPs = plotEl['actualPosition'].values
    elVs = plotEl['actualVelocity'].values
    elXs = plotEl['timestamp'].values - plotEl['timestamp'].values[0] - 2.0
    plotStart = azXs[0]+1.0
    plotEnd = azXs[-1]-1.0
    inPosPlot = inPos[index] - plotAz['timestamp'].values[0] - 2.0 
    InPoss.append(inPosPlot)
    Els.append(elPs[0])
    print(f" {timestamp}, DeltaAz = {delta_az:.2f}, InPosition = {inPosPlot:.2f}")
    
    plotAzXs = np.linspace(azXs[0], azXs[-1], 2000)
    plotElXs = np.linspace(elXs[0], elXs[-1], 2000)
    azPSpline = UnivariateSpline(azXs, azPs, s=0)
    azVelSpline1 = UnivariateSpline(azXs, azVs, s=0) 
    #azVelSpline1 =azPSpline.derivative(n=1)
    # Now smooth the derivative before differentiating again
    smoothedAzVel = np.convolve(azVelSpline1(plotAzXs), kernel, mode='same')
    azVelSpline = UnivariateSpline(plotAzXs, smoothedAzVel, s=smoothingFactor)
    azAccSpline1 = azVelSpline.derivative(n=1)
    smoothedAzAcc = np.convolve(azAccSpline1(plotAzXs), kernel, mode='same')
    # Now smooth the derivative before differentiating again
    azAccSpline = UnivariateSpline(plotAzXs, smoothedAzAcc, s=smoothingFactor)
    azJerkSpline = azAccSpline.derivative(n=1) 
    elPSpline = UnivariateSpline(elXs, elPs, s=0)
    elVelSpline1 = UnivariateSpline(elXs, elVs, s=0)
    #elVelSpline1 =elPSpline.derivative(n=1)
    # Now smooth the derivative before differentiating again
    smoothedElVel = np.convolve(elVelSpline1(plotElXs), kernel, mode='same')
    elVelSpline = UnivariateSpline(plotElXs, smoothedElVel, s=smoothingFactor)
    elAccSpline1 = elVelSpline.derivative(n=1)
    smoothedElAcc = np.convolve(elAccSpline1(plotElXs), kernel, mode='same')
    # Now smooth the derivative before differentiating again
    elAccSpline = UnivariateSpline(plotElXs, smoothedElAcc, s=smoothingFactor)
    elJerkSpline = elAccSpline.derivative(n=1) 

    # Fit the post slew tracking to plot the deviation near the InPosition stamp
    # Calculate the tracking errors
    errorAz = az[(az['timestamp'] > (inPos[index]-1.0)) & (az['timestamp'] < (inPos[index]+5.0))]
    errorEl = el[(el['timestamp'] > (inPos[index]-1.0)) & (el['timestamp'] < (inPos[index]+5.0))]
    fitAz = az[(az['timestamp'] > (inPos[index])) & (az['timestamp'] < (inPos[index]+5.0))]
    fitEl = el[(el['timestamp'] > (inPos[index])) & (el['timestamp'] < (inPos[index]+5.0))]

    error_az_vals = np.array(errorAz['actualPosition'].values)
    error_el_vals = np.array(errorEl['actualPosition'].values)
    error_times_az = errorAz['timestamp'].values
    error_times_el = errorEl['timestamp'].values
    fit_az_vals = np.array(fitAz['actualPosition'].values)
    fit_el_vals = np.array(fitEl['actualPosition'].values)
    fit_times_az = fitAz['timestamp'].values
    fit_times_el = fitEl['timestamp'].values
    # Adjust the time to match the others
    fit_times_az -= start_slew_times[index]
    fit_times_el -= start_slew_times[index]
    error_times_az -= start_slew_times[index]
    error_times_el -= start_slew_times[index]

    # Fit with a polynomial
    az_fit = np.polyfit(fit_times_az, fit_az_vals, 2)
    el_fit = np.polyfit(fit_times_el, fit_el_vals, 2)
    az_model = np.polyval(az_fit, error_times_az)
    el_model = np.polyval(el_fit, error_times_el)

    # Errors in arcseconds
    az_error = (error_az_vals - az_model) * 3600
    el_error = (error_el_vals - el_model) * 3600    
    
    fig.clear()
    plt.subplots_adjust(wspace=0.3, hspace=0.5)
    plt.suptitle(f"MT Mount Slews - {ip_time}", fontsize = 18)
    plt.subplot(5,2,1)
    plt.plot(plotAzXs, azPSpline(plotAzXs), lw=3, color='r', label='Spline fit')
    plt.scatter(azXs, azPs, marker='x', color='red', s=100, label='Measured points')
    plt.plot([0.0,0.0], [np.min(azPs), np.max(azPs)], ls=':', color='b', label="Slew start")
    plt.plot([inPosPlot, inPosPlot], [np.min(azPs), np.max(azPs)], ls=':', color='g', label="In Position")
    plt.title(f"Azimuth\nBlue:Slew Start, Green:InPos\nInPos = {inPosPlot:.2f}")
    plt.ylabel("Degrees")
    plt.xlim(plotStart, plotEnd)
    #plt.legend()
    plt.subplot(5,2,2)
    plt.plot(plotElXs, elPSpline(plotElXs), lw=3, color='g', label='Spline fit')
    plt.scatter(elXs, elPs, marker='x', color='g', s=100, label='Measured points')
    plt.plot([0.0,0.0], [np.min(elPs), np.max(elPs)], ls=':', color='b', label="Slew start")
    plt.plot([inPosPlot, inPosPlot], [np.min(elPs), np.max(elPs)], ls=':', color='g', label="In Position")
    plt.title(f"Elevation\nBlue:Slew Start, Green:InPos\nInPos = {inPosPlot:.2f}")
    plt.ylabel("Degrees")
    plt.xlim(plotStart, plotEnd)
    plt.subplot(5,2,3)
    plt.plot(error_times_az, az_error, lw=3, color='r', label='Tracking error')
    plt.title(f"Azimuth Tracking Error")
    plt.ylabel("Arcseconds")
    plt.xlim(plotStart+4.0, plotEnd+3.0)
    ymin = -0.1
    ymax = 0.1
    plt.plot([inPosPlot, inPosPlot], [ymin, ymax], ls=':', color='g', label="In Position")
    plt.ylim(ymin, ymax)
    #plt.legend()
    plt.subplot(5,2,4)
    plt.plot(error_times_el, el_error, lw=3, color='g', label='Tracking error')
    plt.title(f"Elevation Tracking Error")
    plt.ylabel("Arcseconds")
    plt.xlim(plotStart+4.0, plotEnd+3.0)
    ymin = -0.1
    ymax = 0.1
    plt.plot([inPosPlot, inPosPlot], [ymin, ymax], ls=':', color='g', label="In Position")
    plt.ylim(ymin, ymax)
    #plt.legend()
    plt.subplot(5,2,5)
    plt.plot(plotAzXs, azVelSpline(plotAzXs), lw=3, color='r', label='Spline fit')
    plt.scatter(azXs, azVs, marker='x', color='red', s=100, label='Measured points')
    plt.plot([0.0,0.0], [np.min(azVs), np.max(azVs)], ls=':', color='b', label="Slew start")
    plt.plot([inPosPlot, inPosPlot], [np.min(azVs), np.max(azVs)], ls=':', color='g', label="In Position")
    plt.title(f"Azimuth Velocity")
    plt.ylabel("Degrees/sec")
    plt.xlim(plotStart, plotEnd)
    #plt.legend()
    plt.subplot(5,2,6)
    plt.plot(plotElXs, elVelSpline(plotElXs), lw=3, color='g', label='Spline fit')
    plt.scatter(elXs, elVs, marker='x', color='g', s=100, label='Measured points')
    plt.plot([0.0,0.0], [np.min(elVs), np.max(elVs)], ls=':', color='b', label="Slew start")
    plt.plot([inPosPlot, inPosPlot], [np.min(elVs), np.max(elVs)], ls=':', color='g', label="In Position")
    plt.title(f"Elevation Velocity")
    plt.ylabel("Degrees/sec")
    plt.xlim(plotStart, plotEnd)
    #plt.legend()
    plt.subplot(5,2,7)
    plt.plot(plotAzXs, azAccSpline(plotAzXs), lw=3, color='r', label='Acceleration')
    plt.plot([0.0,0.0], [np.min(azAccSpline(plotAzXs)), np.max(azAccSpline(plotAzXs))], ls=':', color='b', label="Slew start")
    plt.plot([inPosPlot, inPosPlot], [np.min(azAccSpline(plotAzXs)), np.max(azAccSpline(plotAzXs))], ls=':', color='g', label="In Position")
    plt.title(f"Azimuth Acceleration")
    plt.ylabel("Degrees/sec^2")
    plt.xlim(plotStart, plotEnd)
    #plt.legend()
    plt.subplot(5,2,8)
    plt.plot(plotElXs, elAccSpline(plotElXs), lw=3, color='g', label='Acceleration')
    plt.plot([0.0,0.0], [np.min(elAccSpline(plotElXs)), np.max(elAccSpline(plotElXs))], ls=':', color='b', label="Slew start")
    plt.plot([inPosPlot, inPosPlot], [np.min(elAccSpline(plotElXs)), np.max(elAccSpline(plotElXs))], ls=':', color='g', label="In Position")
    plt.title(f"Elevation Acceleration")
    plt.ylabel("Degrees/sec^2")
    plt.xlim(plotStart, plotEnd)
    #plt.legend()
    plt.subplot(5,2,9)
    plt.plot(plotAzXs, azJerkSpline(plotAzXs), lw=3, color='r', label='Jerk')
    plt.plot([0.0,0.0], [np.min(azJerkSpline(plotAzXs)), np.max(azJerkSpline(plotAzXs))], ls=':', color='b', label="Slew start")
    plt.plot([inPosPlot, inPosPlot], [np.min(azJerkSpline(plotAzXs)), np.max(azJerkSpline(plotAzXs))], ls=':', color='g', label="In Position")
    plt.title(f"Azimuth Jerk")
    plt.ylabel("Degrees/sec^3")
    plt.xlim(plotStart, plotEnd)
    #plt.legend()
    plt.subplot(5,2,10)
    plt.plot(plotElXs, elJerkSpline(plotElXs), lw=3, color='g', label='Jerk')
    plt.plot([0.0,0.0], [np.min(elJerkSpline(plotElXs)), np.max(elJerkSpline(plotElXs))], ls=':', color='b', label="Slew start")
    plt.plot([inPosPlot, inPosPlot], [np.min(elJerkSpline(plotElXs)), np.max(elJerkSpline(plotElXs))], ls=':', color='g', label="In Position")
    plt.title(f"Elevation Jerk")
    plt.ylabel("Degrees/sec^3")
    plt.xlim(plotStart, plotEnd)
    #plt.legend()
    timestamp = ip_time.split('.')[0].replace('-','').replace(':','')
    plt.savefig(f"/home/c/cslage/u/MTMount/vel_accel_jerk_plots/MT_Mount_Accel_Jerk_Blowup2_{timestamp}.png")


In [ ]:
good_deltaAzs = []
good_InPoss = []
for i in range(len(deltaAzs)):
    shift = np.sqrt(deltaEls[i]**2 + (deltaAzs[i]*np.cos(Els[i]*np.pi/180.0))**2)
    print(f"{deltaEls[i]:.1f}, {Els[i]:.1f}, {deltaAzs[i]:.1f}, {shift:.2f}")
    if shift > 3.75:
        continue
    else:
        good_deltaAzs.append(deltaAzs[i])
        good_InPoss.append(InPoss[i])

In [ ]:
plt.scatter(good_deltaAzs, good_InPoss)
plt.plot([0,16], [4.0,4.0], ls='--', color='k')
plt.plot([7.0,7.0], [3.0,5.5], ls='--', color='k')
plt.xlim(0,16)
plt.ylim(3.0,5.5)
plt.title("Slew and Settle time vs Azimuth slew")
plt.ylabel("Slew and Settle time (sec)")
plt.xlabel("Azimuth slew (degrees)")
plt.savefig(f"/home/c/cslage/u/MTMount/vel_accel_jerk_plots/Slew_Settle_vs_Az_Slew_06Apr23.png")